### Install dependencies

In [ ]:
%%bash
# git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

## generate ground truth instance labels

In [44]:
import os
import numpy as np
import torch
import glob

os.makedirs("../../dataset/data/replica_features/train", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/val", exist_ok=True)
os.makedirs("../../dataset/data/replica_features/test", exist_ok=True)
train_test_dict = {
    "office0": "train",
    "office1": "train",
    "office2": "train",
    "office3": "val",
    "office4": "test",
    "room0": "train",
    "room1": "val",
    "room2": "test",
}

gt_masks = glob.glob("../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/*.pt")
# gt_instances = glob.glob("dataset/replica_split/ground_truth/*.txt")
for gt_mask_path in gt_masks:
    gt_mask = torch.load(gt_mask_path)[0]
    gt_instance_id_path = gt_mask_path.replace("OpenYOLO3D/output/replica/replica_ground_truth_masks/", "data/replica_split/ground_truth/").replace(".pt", ".txt")
    gt_instance_id = np.loadtxt(gt_instance_id_path, dtype='int')
    assert gt_mask.shape[0] == gt_instance_id.shape[0]
    
    labels = np.zeros(gt_mask.shape[1])
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        instance_label = gt_instance_id[mask_points][0]
        semantics_label = instance_label // 1000
        labels[instance_id] = semantics_label
    scene_name = os.path.splitext(os.path.basename(gt_mask_path))[0]
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_labels"
    np.save(output_path, labels)

## generate instance feature (openscene features and ground truth masks)

In [60]:
# ADJUST PATH!!!
openscene_feature_path = glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/*_features.npy")
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/*_features.npy"))
openscene_feature_path.extend(glob.glob("../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/*_features.npy"))

for feature_path in openscene_feature_path:
    print(feature_path)
    scene_name = os.path.splitext(os.path.basename(feature_path))[0]
    scene_name = scene_name.replace("_features", "")
    gt_mask_path = f"../../dataset/OpenYOLO3D/output/replica/replica_ground_truth_masks/{scene_name}.pt"

    point_features = np.load(feature_path)
    gt_mask = torch.load(gt_mask_path)[0]
    instance_features = np.zeros((gt_mask.shape[1], point_features.shape[1]))
    for instance_id in range(gt_mask.shape[1]):
        mask_points = gt_mask[:, instance_id].bool()
        mask_features = point_features[mask_points, :]
        instance_feature = np.mean(mask_features, axis=0)  # aggregate
        instance_features[instance_id, :] = instance_feature
    split = train_test_dict[scene_name]
    output_path = f"../../dataset/data/replica_features/{split}/{scene_name}_features"
    np.save(output_path, instance_features)
    

../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/room0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office0_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/train/office2_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/room1_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/val/office3_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/office4_features.npy
../../experiments/merged_pipline/run_2025-01-28-00-04-02/openscene/test/room2_features.npy


## Prompt Learning

In [1]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_train_maple_prompt_scene.sh replica 1 ../../dataset/data/

Run this job and save the output to output/base2new/train_base/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1
Error while terminating subprocess (pid=2879446): 


In [3]:
# %%bash
# bash scripts/maple_prompt_scene/base2new_test_maple_prompt_scene.sh replica 1 ../../dataset/data/

Evaluating model
Runing the first phase job and save the output to output/base2new/test_new/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Error while terminating subprocess (pid=2879966): 


In [63]:
os.makedirs("../../dataset/data/replica_features/split_fewshot", exist_ok=True)

In [4]:
%%bash
bash scripts/maple_prompt_scene/xd_train_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets

Run this job and save the output to output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: False
head: 
load_epoch: None
model_dir: 
no_train: False
opts: ['DATASET.NUM_SHOTS', '-1']
output_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_

val: [<datasets.replica.Datum_feature object at 0x7f3ba0332410>, <datasets.replica.Datum_feature object at 0x7f3ba0331990>, <datasets.replica.Datum_feature object at 0x7f3ba0332440>, <datasets.replica.Datum_feature object at 0x7f3ba0332ce0>, <datasets.replica.Datum_feature object at 0x7f3ba03325c0>, <datasets.replica.Datum_feature object at 0x7f3ba03325f0>, <datasets.replica.Datum_feature object at 0x7f3ba0332650>, <datasets.replica.Datum_feature object at 0x7f3ba0332680>, <datasets.replica.Datum_feature object at 0x7f3ba0332d10>, <datasets.replica.Datum_feature object at 0x7f3ba0332530>, <datasets.replica.Datum_feature object at 0x7f3ba0332560>, <datasets.replica.Datum_feature object at 0x7f3ba03326b0>, <datasets.replica.Datum_feature object at 0x7f3ba0332c80>, <datasets.replica.Datum_feature object at 0x7f3ba0332a70>, <datasets.replica.Datum_feature object at 0x7f3ba0332bf0>, <datasets.replica.Datum_feature object at 0x7f3ba03328f0>, <datasets.replica.Datum_feature object at 0x7f3ba0

-----------------------------------------------------------
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7f3ba0331a20>, <datasets.replica.Datum_feature object at 0x7f3ba0330f10>, <datasets.replica.Datum_feature object at 0x7f3ba0331240>, <datasets.replica.Datum_feature object at 0x7f3ba0330df0>, <datasets.replica.Datum_feature object at 0x7f3ba0331750>, <datasets.replica.Datum_feature object at 0x7f3ba0331930>, <datasets.replica.Datum_feature object at 0x7f3ba0331840>, <datasets.replica.Datum_feature object at 0x7f3ba0331870>, <datasets.replica.Datum_feature object at 0x7f3ba03311e0>, <datasets.replica.Datum_feature object at 0x7f3ba0331660>, <datasets.replica.Datum_feature object at 0x7f3ba0331780>, <datasets.replica.Datum_feature object at 0x7f3ba0331570>, <datasets.replica.Datum_feature object at 0x7f3ba0330fd0>, <datasets.replica.Datum_feature object at 0x7f3ba0331180>, <datasets.replica.Datum_feature object at 0x7f

...

=> result
* total: 106
* correct: 49
* accuracy: 46.2%
* error: 53.8%
* macro_f1: 31.6%
epoch [1000/1000] batch [20/44] time 0.036 (0.055) data 0.000 (0.014) loss 0.8169 (0.5173) lr 3.4544e-08 eta 0:00:01
epoch [1000/1000] batch [40/44] time 0.038 (0.047) data 0.000 (0.007) loss 0.5620 (0.5268) lr 3.4544e-08 eta 0:00:00
Evaluate on the *val* set


100%|██████████| 2/2 [00:00<00:00,  5.15it/s]


=> result
* total: 106
* correct: 49
* accuracy: 46.2%
* error: 53.8%
* macro_f1: 31.7%
Checkpoint saved to output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/MultiModalPromptSceneLearner/model.pth.tar-1000
Finish training
Deploy the model with the best val performance

Loading weights to MultiModalPromptSceneLearner from output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/MultiModalPromptSceneLearner/model-best.pth.tar (epoch = 477)
Number of deep prompt layers: 11
Evaluate on the *test* set


100%|██████████| 1/1 [00:00<00:00,  2.61it/s]


=> result
* total: 83
* correct: 46
* accuracy: 55.4%
* error: 44.6%
* macro_f1: 33.1%
Elapsed: 0:42:41


In [1]:
from reconstruct_prompts import PromptReconstructor
import torch
#
## Define paths
prompt_path = "output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/learned_prompts/learned_prompts_final.pt"
output_path = "replica_prompts.pt"

# Define class names
class_names = ["basket", "bed", "bench", "bin", "blanket", "blinds", "book", "bottle", "box", "bowl", "camera", "cabinet", "candle", "chair", "clock", "cloth", "comforter", "cushion", "desk", "desk-organizer", "door", "indoor-plant", "lamp", "monitor", "nightstand", "panel", "picture", "pillar", "pillow", "pipe", "plant-stand", "plate", "pot", "sculpture", "shelf", "sofa", "stool", "switch", "table", "tablet", "tissue-paper", "tv-screen", "tv-stand", "vase", "vent", "wall-plug", "window", "rug"]

# Create the reconstructor
reconstructor = PromptReconstructor(prompt_path, class_names)

# You can examine intermediate results if you want
complete_prompts, compound_prompts = reconstructor.reconstruct_complete_prompts()
print("Complete prompts shape:", complete_prompts.shape)
print("Number of compound prompt layers:", len(compound_prompts))

# Get the encoded prompts
encoded_prompts = reconstructor.encode_prompts()
print("\nEncoded prompts shape:", encoded_prompts.shape)
print("Encoded prompts statistics:")
print(f"Mean: {encoded_prompts.mean().item():.6f}")
print(f"Std: {encoded_prompts.std().item():.6f}")

# Save the final results
reconstructor.save_encoded_prompts(output_path)

/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Complete prompts shape: torch.Size([48, 77, 512])
Number of compound prompt layers: 11

Encoded prompts shape: torch.Size([48, 512])
Encoded prompts statistics:
Mean: 0.001771
Std: 0.044159
Saved encoded prompts to replica_prompts.pt
Shape: torch.Size([48, 512])
Number of classes: 48


In [13]:
from inspect_prompts import inspect_prompt_file
import torch

# output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/learned_prompts/learned_prompts_final.pt

prompts = inspect_prompt_file("output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/learned_prompts/learned_prompts_final.pt")
ctx = prompts['ctx']
correlation = torch.corrcoef(ctx)
print("\nContext vector correlations:")
print(correlation)


Inspecting prompts from: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1/learned_prompts/learned_prompts_final.pt

=== Component Shapes ===
ctx: shape=torch.Size([4, 512]), dtype=torch.float16, device=cuda:0
token_prefix: shape=torch.Size([48, 1, 512]), dtype=torch.float16, device=cuda:0
token_suffix: shape=torch.Size([48, 72, 512]), dtype=torch.float16, device=cuda:0
n_ctx: <class 'int'>
n_cls: <class 'int'>
compound_prompts_text: 11 tensors of shape torch.Size([4, 512])
tokenized_prompts: shape=torch.Size([48, 77]), dtype=torch.int64, device=cpu

=== Context Vector Analysis ===
Statistics:
  Mean: 0.000149
  Std:  0.053589
  Min:  -0.328369
  Max:  0.254883

=== Compound Prompts Analysis ===

Layer 0:
  Shape: torch.Size([4, 512])
  Mean:  -0.000305
  Std:   0.098565
  Range: [-0.491092, 0.527134]

Layer 1:
  Shape: torch.Size([4, 512])
  Mean:  0.000472
  Std:   0.116268
  Range: [-0.488564, 0.473930]

Layer 2:
  Shape: torch.Size([

In [3]:
# !rm -r output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1

In [2]:
%%bash
bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../dataset/data/ vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets

Run this job and save the output to output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/replica/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: True
head: 
load_epoch: None
model_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/seed1
no_train: False
opts: []
output_dir: output/evaluation/MaPLePromptScene/vit_b16_c2_ep200_batch4_ctx4_depth12_cross_datasets_-1shots/replica/seed1
resume: 
root: ../../dataset/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPL

val: [<datasets.replica.Datum_feature object at 0x7f314deae290>, <datasets.replica.Datum_feature object at 0x7f314deac7f0>, <datasets.replica.Datum_feature object at 0x7f314deae2c0>, <datasets.replica.Datum_feature object at 0x7f314deaeb30>, <datasets.replica.Datum_feature object at 0x7f314deae440>, <datasets.replica.Datum_feature object at 0x7f314deae470>, <datasets.replica.Datum_feature object at 0x7f314deae4d0>, <datasets.replica.Datum_feature object at 0x7f314deae500>, <datasets.replica.Datum_feature object at 0x7f314deaea40>, <datasets.replica.Datum_feature object at 0x7f314deae3b0>, <datasets.replica.Datum_feature object at 0x7f314deae3e0>, <datasets.replica.Datum_feature object at 0x7f314deae530>, <datasets.replica.Datum_feature object at 0x7f314deaeb60>, <datasets.replica.Datum_feature object at 0x7f314deae8f0>, <datasets.replica.Datum_feature object at 0x7f314deaeb00>, <datasets.replica.Datum_feature object at 0x7f314deae770>, <datasets.replica.Datum_feature object at 0x7f314d

-----------------------------------------------------------
-----------------------------------------------------------
train: [<datasets.replica.Datum_feature object at 0x7f314dead810>, <datasets.replica.Datum_feature object at 0x7f314deacac0>, <datasets.replica.Datum_feature object at 0x7f314dead780>, <datasets.replica.Datum_feature object at 0x7f314deace80>, <datasets.replica.Datum_feature object at 0x7f314dead7b0>, <datasets.replica.Datum_feature object at 0x7f314dead570>, <datasets.replica.Datum_feature object at 0x7f314dead6f0>, <datasets.replica.Datum_feature object at 0x7f314dead6c0>, <datasets.replica.Datum_feature object at 0x7f314dead120>, <datasets.replica.Datum_feature object at 0x7f314deaca30>, <datasets.replica.Datum_feature object at 0x7f314dead960>, <datasets.replica.Datum_feature object at 0x7f314dead540>, <datasets.replica.Datum_feature object at 0x7f314deacdf0>, <datasets.replica.Datum_feature object at 0x7f314dead210>, <datasets.replica.Datum_feature object at 0x7f

val: [<datasets.replica.Datum_feature object at 0x7f314deae290>, <datasets.replica.Datum_feature object at 0x7f314deac7f0>, <datasets.replica.Datum_feature object at 0x7f314deae2c0>, <datasets.replica.Datum_feature object at 0x7f314deaeb30>, <datasets.replica.Datum_feature object at 0x7f314deae440>, <datasets.replica.Datum_feature object at 0x7f314deae470>, <datasets.replica.Datum_feature object at 0x7f314deae4d0>, <datasets.replica.Datum_feature object at 0x7f314deae500>, <datasets.replica.Datum_feature object at 0x7f314deaea40>, <datasets.replica.Datum_feature object at 0x7f314deae3b0>, <datasets.replica.Datum_feature object at 0x7f314deae3e0>, <datasets.replica.Datum_feature object at 0x7f314deae530>, <datasets.replica.Datum_feature object at 0x7f314deaeb60>, <datasets.replica.Datum_feature object at 0x7f314deae8f0>, <datasets.replica.Datum_feature object at 0x7f314deaeb00>, <datasets.replica.Datum_feature object at 0x7f314deae770>, <datasets.replica.Datum_feature object at 0x7f314d

-----------------------------------------------------------
Label range: 0-47
Unique labels: {0, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47}
Label range: 0-47
Unique labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 21, 22, 24, 25, 26, 28, 29, 31, 35, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47}
Label range: 0-0
Unique labels: {0}
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
---------  -------
Dataset    Replica
# classes  48
# train_x  178
# val      106
# test     192
---------  

100%|██████████| 2/2 [00:00<00:00,  2.05it/s]



=== Starting inference forward pass ===

=== Inspecting Input point features ===
Shape: torch.Size([100, 512])
Type: torch.float16
Device: cuda:0
Stats: min=-7.511719, max=19.531250, mean=0.037354

=== Inspecting Generated prompts ===
Shape: torch.Size([45, 77, 512])
Type: torch.float16
Device: cuda:0
Stats: min=-0.442139, max=0.463379, mean=0.000152

=== Inspecting Text features ===
Shape: torch.Size([45, 512])
Type: torch.float16
Device: cuda:0
Stats: min=-0.996094, max=3.291016, mean=0.004101

=== Inspecting Final logits ===
Shape: torch.Size([100, 45])
Type: torch.float16
Device: cuda:0
Stats: min=-28.656250, max=16.062500, mean=-6.285156
input: tensor([[ 0.1658,  0.4202, -0.0504,  ..., -1.3438,  0.0606,  0.3818],
        [-0.1439, -0.2399, -0.0358,  ..., -0.3845, -0.2786, -0.0889],
        [-0.1652, -0.0839, -0.1644,  ..., -0.3892, -0.2759, -0.0550],
        ...,
        [ 0.1648,  0.4475, -0.0604,  ..., -1.0439,  0.0222,  0.2222],
        [-0.1808, -0.3528, -0.0277,  ..., -0.424